I am going to build a Large Language model and yeah ..that's it

#1.EDA


In [4]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.9.0+cpu
tiktoken version: 0.12.0


In [5]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [6]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [7]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

class TextdatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
        return len(self.input_ids)

  def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [8]:
 def __len__(self):
        return len(self.input_ids)

 def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [9]:

def create_dataloader_v1(txt, batch_size, max_length, stride,
                         shuffle=True, drop_last=True, num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = TextdatasetV1atasetV1(txt, tokenizer, max_length, stride)


    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


In [10]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

vocab_size = 50257
output_dim = 256
context_length = 1024


token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

def create_dataloader_v1(txt, batch_size, max_length, stride,
                         shuffle=True, drop_last=True, num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")

    # Corrected typo here
    dataset = TextdatasetV1(txt, tokenizer, max_length, stride)


    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

batch_size = 8
max_length = 4
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length
)

In [11]:

 for batch in dataloader:
  x,y = batch

  token_embeddingd=token_embedding_layer(x)
  pos_embeddings=pos_embedding_layer(torch.arange(max_length))

  input_embeddings=token_embeddingd+pos_embeddings

  break

In [12]:
print(input_embeddings.shape)

torch.Size([8, 4, 256])


#1.Attention Mechanisms

multi head attention

In [16]:
import torch
import torch.nn as nn # Import torch.nn as nn

class CausalSelfAttention(nn.Module): # Corrected to nn.Module
  def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
    super().__init__() # Missing super().__init__()
    self.d_out = d_out
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias) # Corrected to W_key
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.dropout = nn.Dropout(dropout)
    # Mask definition for causal attention
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1).bool())

  def forward(self, x):
    b, n_tokens, d_in = x.shape
    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    attn_scores = queries @ keys.transpose(1, 2) # (b, n_tokens, n_tokens)
    attn_scores.masked_fill_(self.mask[:n_tokens, :n_tokens], -torch.inf)
    attn_weights = torch.softmax(attn_scores / (keys.shape[-1]**0.5), dim=-1)
    attn_weights = self.dropout(attn_weights)

    context_vec = attn_weights @ values
    return context_vec # Corrected variable name

class MultiHeadAttention(nn.Module): # Corrected class name and nn.Module
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()
    self.num_heads = num_heads # Store num_heads
    self.d_out = d_out # Store d_out for output projection
    # Each head processes d_in, but output d_out/num_heads. Assuming d_out is total output dim.
    # Let's adjust CausalSelfAttention to take d_in for its outputs to be consistent.
    # If d_out is the dimension per head, then d_in should be passed to CausalSelfAttention
    # If d_out is the total output dimension, then each head should output d_out // num_heads
    # For simplicity, let's assume d_in is passed to CausalSelfAttention and its output will be concatenated
    # Then, d_out_per_head should be d_out
    self.heads = nn.ModuleList(
        [CausalSelfAttention(d_in, d_out, context_length, dropout, qkv_bias)
         for _ in range(num_heads)] # Corrected for loop and variable name
    )
    self.out_proj = nn.Linear(num_heads * d_out, d_out) # Added output projection layer

  def forward(self, x):
    # Each head outputs d_out, so concatenation results in num_heads * d_out
    context_vecs = torch.cat([head(x) for head in self.heads], dim=-1)
    # Project the concatenated output back to d_out
    return self.out_proj(context_vecs)


In [19]:
torch.manual_seed(123)

d_in = output_dim
d_out = output_dim
num_heads = 8
context_length = max_length
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads)

batch = input_embeddings
context_vecs = mha(batch)

print("context_vecs.shape:", context_vecs.shape)


context_vecs.shape: torch.Size([8, 4, 256])
